<A HREF="https://theaidigest.in/topic-modeling-using-roberta-and-transformers/">Topic modeling using Roberta and transformers</A><BR>
    <A HREF="https://maartengr.github.io/BERTopic/index.html">BERTopic documentation</A>

In [ ]:
!pip3 install bertopic

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import en_core_web_md
import csv
import regex as re
import os
import spacy
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from gensim import corpora
import pprint

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook() #Notebook visualisation enabled

from bertopic import BERTopic # Much faster with GPU than on CPU
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

from sentence_transformers import SentenceTransformer

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

# to add words to sklearn's countvectorizer stopword list
from sklearn.feature_extraction import text

c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\plotly\graph_objs\__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\plotly\graph_objs\__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\plotly\graph_objs\__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\plotly\graph_objs\__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version inste

In [2]:
# Load up the files
#path = './DataUCSB/' # Smaller UCSB dataset
path = './Data/' # larger American Rhetoric dataset
list_of_files = []

for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.txt'):
            list_of_files.append(os.path.join(root,file))

filepath = './DataUCSB/address-before-joint-session-the-congress-the-state-the-union-16.csv'
speeches = []
for file in list_of_files:
    with open(file, encoding='utf-8') as f:
        text = f.read()
    f.close()
    speeches.append(text)

#clean out xa0 space characters
[speech.replace(u'\xa0', '') for speech in speeches]; # ; supresses output

<A HREF="https://datagy.io/python-flatten-list-of-lists">Flatten list of lists</A><BR>

In [3]:
listlist = [speech.split('.') for speech in speeches]

# Flatten list template: [item for sublist in list_of_lists for item in sublist]
# Here item is sentence, sublist is speech, list_of_lists is listlist
sentences = [sentence.lstrip().replace(u'\xa0', '') for speech in listlist for sentence in speech if len(sentence) > 0]

<A HREF="https://www.sbert.net/#">SBERT SentenceTransformers</A>

In [4]:
# Prepare embeddings - this takes about 5-10 minutes and gets the GPU warmed up!
# all-mpnet-base-v2 is about 400MB
sentence_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = sentence_model.encode(sentences)

# Sentence Embeddings done!
#  now skip a few cells to where there is a nice big comment

In [93]:
sentences[122]

'When Richard came to the Oval Office, we talked about what those automatic budget cuts -- sequestration -- could mean for the VA'

In [94]:
embeddings[122]

array([ 3.46503104e-03,  3.14406492e-02, -5.08073671e-03, -3.96114737e-02,
       -1.16334185e-02, -2.96727717e-02, -5.42856678e-02,  8.92422348e-03,
       -7.73201063e-02, -4.46461141e-02,  5.16621023e-02,  2.76710652e-02,
        5.11454600e-05,  1.42952045e-02, -2.27853954e-02,  1.93038199e-03,
        6.41904548e-02, -3.04587297e-02, -1.52980909e-02, -2.19308771e-02,
       -5.57330204e-03,  1.19999601e-02,  2.57524243e-03,  2.12978777e-02,
        2.11541876e-02, -3.59955477e-03,  4.11274144e-03,  9.29337740e-03,
       -6.88223122e-03, -5.48311509e-03,  7.84348976e-03,  1.76475253e-02,
        3.47615480e-02, -1.13796256e-01,  1.93099117e-06, -1.03333062e-02,
        1.30778717e-04,  5.71332872e-03, -1.01478718e-01, -3.00557483e-02,
        2.07122006e-02, -6.32899627e-02,  7.57661462e-03, -1.33303087e-02,
        2.91231135e-03, -4.23367228e-03, -1.11689391e-02, -3.08362693e-02,
        1.57363303e-02, -4.59545897e-03, -8.12980230e-04,  1.75165571e-02,
       -5.43145090e-02, -

In [4]:
#import nltk
#from nltk.corpus import stopwords

In [5]:
#Load SpaCy English Model
#nlp = en_core_web_md.load()

#Tags to remove
#extags = ['PRON','CCONJ','PUNCT','PART','DET','ADP','NUM','SYM','SPACE']

#tokens=[]
#SpaCy tokenization + lemmatization + lowercase
#for speech in nlp.pipe(speeches):
#    scr_tok = [token.lemma_.lower() for token in speech if token.pos_ not in extags and not token.is_stop and token.is_alpha]
#    tokens.append(scr_tok)

<A HREF="https://maartengr.github.io/BERTopic/getting_started/tips_and_tricks/tips_and_tricks.html#removing-stop-words">Removing stop words</A> a little different than sometimes
<P>
At times, stop words might end up in our topic representations. This is something we typically want to avoid as they contribute little to the interpretation of the topics. However, removing stop words as a preprocessing step is not advised as the transformer-based embedding models that we use need the full context in order to create accurate embeddings.
</P>
<P>
Instead, we can use the CountVectorizer to preprocess our documents after having generated embeddings and clustered our documents. Personally, I have found almost no disadvantages to using the CountVectorizer to remove stopwords and it is something I would strongly advise to try out:
</P>

<A HREF="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html">sklearn CountVectorizer docs</A>

In [97]:
################# RUN THIS AFTER SENTENCE EMBEDDINGS ###################
# This takes a minute or two, shorter than above
stop_words = text.ENGLISH_STOP_WORDS.union(['thats', 'just', 'im', 'did', 'thing', 'mr', 'al', 'thank', 'okay', 'thanks', 'question'])
#vectorizer_model = CountVectorizer(stop_words='english', ngram_range=(1,3))
vectorizer_model = CountVectorizer(stop_words=stop_words, ngram_range=(1,3))
model = BERTopic(vectorizer_model=vectorizer_model, nr_topics=15)
topics, probs = model.fit_transform(sentences, embeddings)

In [98]:
model.get_topic_info()

,Topic,Count,Name
0,-1,73349,-1_people_going_make_know
1,0,2214,0_health_care_insurance_health care
2,1,1580,1_nuclear_iran_weapons_sanctions
3,2,1085,2_congress_senate_republicans_bipartisan
4,3,852,3_president_president president_obama_presiden...
5,4,743,4_right_true_right right_young lady
6,5,628,5_israel_palestinians_israels_israelis
7,6,578,6_governor_romney_governor romney_ms
8,7,572,7_afghanistan_afghan_troops_iraq
9,8,548,8_cuba_cuban_cuban people_cubans


The -1 topic is an outlier topic and contains documents that are generally unrelated to one another. The documents in that topic are essentially random in nature and therefore do not represent an actual topic. Thus, the -1 topic cannot be interpreted as all other topics. <A HREF="https://github.com/MaartenGr/BERTopic/issues/677">reference</A>

In [67]:
model.get_topic(0)

[('health', 0.05743176192986177),
 ('care', 0.04968332592662061),
 ('insurance', 0.04799788859926906),
 ('health care', 0.039829151255246255),
 ('health insurance', 0.023812967195562845),
 ('medicare', 0.01807584033017865),
 ('costs', 0.015565726202935826),
 ('people', 0.015555627819126698),
 ('affordable', 0.015325133189273666),
 ('coverage', 0.014518833270966312)]

In [107]:
model.get_topics()

{-1: [('people', 0.010377612906681204),
  ('going', 0.007641722375852682),
  ('make', 0.006784588271381897),
  ('know', 0.006144650102223625),
  ('think', 0.006119250472113379),
  ('want', 0.006080408040338213),
  ('president', 0.006072376561537916),
  ('world', 0.006019679118082326),
  ('like', 0.005879196322329415),
  ('weve', 0.005649466781900936)],
 0: [('health', 0.05803863125973147),
  ('care', 0.05051548908254219),
  ('insurance', 0.047716179074347706),
  ('health care', 0.040178550113369065),
  ('health insurance', 0.023945236359090027),
  ('medicare', 0.018183646898372544),
  ('affordable', 0.015372234813416168),
  ('people', 0.015291624693165876),
  ('costs', 0.015228378211031887),
  ('coverage', 0.014164132671861066)],
 1: [('nuclear', 0.07177077407580301),
  ('iran', 0.061960487986230904),
  ('weapons', 0.02437270120221643),
  ('sanctions', 0.023082420766392067),
  ('iranian', 0.02186307434905729),
  ('nuclear weapons', 0.021213822794592548),
  ('deal', 0.0188400329616135),

In [69]:
model.get_topic(0)

[('health', 0.05743176192986177),
 ('care', 0.04968332592662061),
 ('insurance', 0.04799788859926906),
 ('health care', 0.039829151255246255),
 ('health insurance', 0.023812967195562845),
 ('medicare', 0.01807584033017865),
 ('costs', 0.015565726202935826),
 ('people', 0.015555627819126698),
 ('affordable', 0.015325133189273666),
 ('coverage', 0.014518833270966312)]

In [70]:
model.visualize_topics()

c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [71]:
# Save Model
model.save("obamaspeechmodel")
# Saved model is about 1GB... may not push to github, we'll see, maybe save it outside repository or add to .ignore

c:\users\peter\appdata\local\programs\python\python39\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
# Load Model
model = BERTopic.load("obamaspeechmodel")

In [103]:
testfile = './NYTimes/12-14-2012-sandyhook.txt'
with open(testfile, encoding='utf-8') as f:
        test = f.read()
f.close()
test = test.replace("\n\n", "")
test = test.split('.')
testembed = sentence_model.encode(test)

In [104]:
testtopics, testprobs = model.transform(test, testembed)

In [108]:
[model.get_topic(topic) for topic in testtopics if topic>=0]
# There isn't a gun control topic so this is what it came up with when it wasn't spitting out -1 (outlier)
# work/responsibility and health care

[[('health', 0.05803863125973147),
  ('care', 0.05051548908254219),
  ('insurance', 0.047716179074347706),
  ('health care', 0.040178550113369065),
  ('health insurance', 0.023945236359090027),
  ('medicare', 0.018183646898372544),
  ('affordable', 0.015372234813416168),
  ('people', 0.015291624693165876),
  ('costs', 0.015228378211031887),
  ('coverage', 0.014164132671861066)],
 [('everybody', 0.20885369814405264),
  ('work', 0.10816418040998534),
  ('everybody everybody', 0.10157083071862132),
  ('responsibility', 0.06673583707638442),
  ('got', 0.06345553356342755),
  ('work everybody', 0.05739625180135082),
  ('weve got', 0.05375973733426491),
  ('weve', 0.04360333841920299),
  ('weve got work', 0.03928771455214753),
  ('got work', 0.03817819866955677)],
 [('everybody', 0.20885369814405264),
  ('work', 0.10816418040998534),
  ('everybody everybody', 0.10157083071862132),
  ('responsibility', 0.06673583707638442),
  ('got', 0.06345553356342755),
  ('work everybody', 0.05739625180135